## Import libraries

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rasterio
import pyproj
from rasterio.crs import CRS
import cartopy.crs as ccrs
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.ticker as mticker
from importlib import reload
from pystac_client import Client
import rivabar as rb

%matplotlib qt
plt.rcParams['svg.fonttype'] = 'none'

## Branco River, Brazil

You can find the links to the Landsat bands that are needed by running this cell. Clicking the links will download the raster files.

In [326]:
catalog = Client.open("https://landsatlook.usgs.gov/stac-server")
scene_id = "LC82320602014050LGN02"
search = catalog.search(
    collections=["landsat-c2l2-sr"],
    query={"landsat:scene_id": {"eq": scene_id}},
    limit=1
)
item = next(search.items())
for band in ['red', 'green', 'blue', 'swir16']:
    print(item.assets[band].href)

https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2014/232/060/LC08_L2SP_232060_20140219_20200911_02_T1/LC08_L2SP_232060_20140219_20200911_02_T1_SR_B4.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2014/232/060/LC08_L2SP_232060_20140219_20200911_02_T1/LC08_L2SP_232060_20140219_20200911_02_T1_SR_B3.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2014/232/060/LC08_L2SP_232060_20140219_20200911_02_T1/LC08_L2SP_232060_20140219_20200911_02_T1_SR_B2.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2014/232/060/LC08_L2SP_232060_20140219_20200911_02_T1/LC08_L2SP_232060_20140219_20200911_02_T1_SR_B6.TIF


### Create water index image and pick start- and end points for channel belt

In [26]:
dirname = '../data/Branco/' # folder with data for the Branco River
fname = 'LC08_L2SP_232060_20140219_20200911_02_T1_SR' # this is a folder that contains the Landsat bands needed to compute the mndwi image
file_type = 'multiple_tifs'
mndwi, left_utm_x, upper_utm_y, right_utm_x, lower_utm_y, delta_x, delta_y, dataset = rb.create_mndwi(dirname, fname, 
            file_type, mndwi_threshold=0.01, delete_pixels_polys=False, small_hole_threshold=16, remove_smaller_components=False, solidity_filter=False)
plt.figure(figsize=(10, 10))
plt.imshow(mndwi, extent=[left_utm_x, right_utm_x, lower_utm_y, upper_utm_y], cmap='gray_r');

reading Landsat data


100%|██████████| 4/4 [00:00<00:00,  4.94it/s]


removing small holes
removing small components


In [ ]:
# Get two points from user input (these do not need to be exactly on the channel)
points = plt.ginput(n=2)
# Plot start point in green with 'start' label
plt.plot(points[0][0], points[0][1], 'go', markersize=5)
plt.text(points[0][0]+50, points[0][1]+50, 'start', fontsize=10, verticalalignment='bottom')
# Plot end point in red with 'end' label
plt.plot(points[1][0], points[1][1], 'ro', markersize=5)
plt.text(points[1][0]+50, points[1][1]+50, 'end', fontsize=10, verticalalignment='bottom')
plt.draw()
start_x = points[0][0]
start_y = points[0][1]
end_x = points[1][0]
end_y = points[1][1]
print(start_x, start_y, end_x, end_y)

675779.5909090908 98044.94805194804 628489.9675324674 -91715.96103896103


### Extract channel centerlines and banklines

In [28]:
# start_x = 675796.2
# start_y = 98338.8
# end_x = 628190.3
# end_y = -91886.6

D_primal, G_rook, G_primal, mndwi, dataset, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y, xs, ys = \
        rb.extract_centerline(fname, dirname, start_x, start_y, end_x, end_y,\
        file_type='multiple_tifs', flip_outlier_edges=True, mndwi_threshold=0.0,\
        ch_belt_smooth_factor=1e8, ch_belt_half_width=2000, remove_smaller_components=True,\
        delete_pixels_polys=False, small_hole_threshold=64, solidity_filter=False, plot_D_primal=True)

reading Landsat data


100%|██████████| 4/4 [00:00<00:00,  5.01it/s]


removing small holes
removing small components
running skeletonization
building graph from skeleton
finding reasonable starting and ending points on graph edges
finding nodes that are within a certain radius of the path


100%|██████████| 65/65 [00:00<00:00, 5987.39it/s]


creating linestrings for primal graph


...

100%|██████████| 67/67 [00:21<00:00,  3.11it/s]


creating directed graph


100%|██████████| 192/192 [00:01<00:00, 132.96it/s]


getting bank coordinates for main channel


100%|██████████| 192/192 [00:00<00:00, 1334.31it/s]


### Plot width data for main channel

In [337]:
# plot width data for main channel
xl, yl, w1l, w2l, w, s = rb.get_channel_widths_along_path(D_primal, D_primal.graph['main_path'])
plt.figure(figsize=(12, 4))
plt.plot(s, (np.array(w1l)+np.array(w2l))*30.0)
plt.xlabel('along-channel distance (m)')
plt.ylabel('channel width (m)');

### Plot banks and centerline for main channel

In [338]:
x, y, x_utm1, y_utm1, x_utm2, y_utm2 = rb.get_bank_coords_for_main_channel(D_primal, mndwi, D_primal.graph['main_path'], dataset)
plt.figure()
plt.plot(x, y)
plt.plot(x_utm1, y_utm1)
plt.plot(x_utm2, y_utm2)
plt.axis('equal');

100%|██████████| 6786/6786 [00:00<00:00, 28938.53it/s]


### Save results to shapefiles

In [118]:
reload(rb)
rb.write_shapefiles_and_graphs(G_rook, D_primal, dataset, dirname, 'Branco')

### Display an image band in UTM coordinates

The cell below is an example of how you can load the first band from a raster file and display it in UTM coordinate space (so that you can use 'ginput' to get start_x, start_y, end_x, end_y).

In [340]:
reload(rb)
dirname = '../data/Branco/LC08_L2SP_232060_20140219_20200911_02_T1_SR/'
fname = 'LC08_L2SP_232060_20140219_20200911_02_T1_SR_B2.tif'
im, dataset, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y = rb.read_and_plot_im(dirname, fname)

## Mamore River, Bolivia


Data download links:

In [341]:
scene_id = "LC82320712016168LGN01"
search = catalog.search(
    collections=["landsat-c2l2-sr"],
    query={"landsat:scene_id": {"eq": scene_id}},
    limit=1
)
item = next(search.items())
for band in ['red', 'green', 'blue', 'swir16']:
    print(item.assets[band].href)

https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2016/232/071/LC08_L2SP_232071_20160616_20200906_02_T1/LC08_L2SP_232071_20160616_20200906_02_T1_SR_B4.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2016/232/071/LC08_L2SP_232071_20160616_20200906_02_T1/LC08_L2SP_232071_20160616_20200906_02_T1_SR_B3.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2016/232/071/LC08_L2SP_232071_20160616_20200906_02_T1/LC08_L2SP_232071_20160616_20200906_02_T1_SR_B2.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2016/232/071/LC08_L2SP_232071_20160616_20200906_02_T1/LC08_L2SP_232071_20160616_20200906_02_T1_SR_B6.TIF


In [422]:
reload(rb)
dirname = '../data/Mamore/'
fname = 'LC08_L2SP_232071_20160616_20200906_02_T1_SR'
start_x = 303810
start_y = -1851620
end_x = 285603
end_y = -1666131

# note that 'flip_outlier_edges' and 'check_edges' are both set to 'False' for a meandering river
D_primal, G_rook, G_primal, mndwi, dataset, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y, xs, ys = \
        rb.extract_centerline(fname, dirname, start_x, start_y, end_x, end_y,\
        file_type='multiple_tifs', ch_belt_smooth_factor=1e8, ch_belt_half_width=2000, remove_smaller_components=False,\
        delete_pixels_polys=False, small_hole_threshold=64, solidity_filter=False, plot_D_primal=True, flip_outlier_edges=False, check_edges=False)

reading Landsat data


100%|██████████| 4/4 [00:00<00:00,  4.67it/s]


removing small holes
removing small components
running skeletonization
building graph from skeleton
finding reasonable starting and ending points on graph edges
no path between start_ind and end_ind
finding nodes that are within a certain radius of the path


100%|██████████| 12/12 [00:00<00:00, 5744.31it/s]


creating linestrings for primal graph


...

100%|██████████| 14/14 [00:10<00:00,  1.37it/s]


creating directed graph


100%|██████████| 37/37 [00:00<00:00, 58.82it/s]


getting bank coordinates for main channel


100%|██████████| 37/37 [00:00<00:00, 746.27it/s]


### Plot width data for main channel

In [347]:
xl, yl, w1l, w2l, w, s = rb.get_channel_widths_along_path(D_primal, D_primal.graph['main_path'])
plt.figure(figsize=(12, 4))
plt.plot(s, np.array(w)*30.0)
plt.xlabel('along-channel distance (m)')
plt.ylabel('channel width (m)');

### Plot meander wavelength and channel width data

In [348]:
fig, ax = plt.subplots()
plt.imshow(mndwi, extent=[left_utm_x, right_utm_x, lower_utm_y, upper_utm_y], cmap='gray_r', alpha=0.5)
df, curv, s, loc_zero_curv, xsmooth, ysmooth = rb.analyze_width_and_wavelength(D_primal, D_primal.graph['main_path'], ax, delta_s=5, smoothing_factor=0.5*1e7, min_sinuosity=1.1, dx=30)
df.head()

,wavelengths (m),sinuosities,mean widths (m),std. dev. of widths (m),along-channel distance (km)
0,5431.627861,1.211017,204.999671,60.589707,2.283192
1,2558.965193,2.044979,146.129310,29.259071,8.296027
2,4589.684399,1.220040,188.131695,47.769163,11.860503
3,5460.337881,1.728385,206.057214,45.262533,16.967195
4,3392.622177,1.295413,225.374885,53.019634,20.441368


## Brahmaputra River, India / Bangladesh


Data download links:

In [323]:
catalog = Client.open("https://landsatlook.usgs.gov/stac-server")
scene_id = "LC81380422019014LGN00"
search = catalog.search(
    collections=["landsat-c2l2-sr"],
    query={"landsat:scene_id": {"eq": scene_id}},
    limit=1
)
item = next(search.items())
for band in ['red', 'green', 'blue', 'swir16']:
    print(item.assets[band].href)

https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2019/138/042/LC08_L2SP_138042_20190114_20200830_02_T1/LC08_L2SP_138042_20190114_20200830_02_T1_SR_B4.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2019/138/042/LC08_L2SP_138042_20190114_20200830_02_T1/LC08_L2SP_138042_20190114_20200830_02_T1_SR_B3.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2019/138/042/LC08_L2SP_138042_20190114_20200830_02_T1/LC08_L2SP_138042_20190114_20200830_02_T1_SR_B2.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2019/138/042/LC08_L2SP_138042_20190114_20200830_02_T1/LC08_L2SP_138042_20190114_20200830_02_T1_SR_B6.TIF


In [427]:
reload(rb)
dirname = '../data/Brahmaputra/'
fname = 'LC08_L2SP_138042_20190114_20200830_02_T1_SR' 
start_x = 859250.5
start_y = 2903119.2
end_x = 767358.1
end_y = 2776067.9

# setting 'flip_outlier_edges=True' is important in this case
D_primal, G_rook, G_primal, mndwi, dataset, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y, xs, ys = \
        rb.extract_centerline(fname, dirname, start_x, start_y, end_x, end_y,\
        file_type='multiple_tifs', ch_belt_smooth_factor=1e8, ch_belt_half_width=2000, remove_smaller_components=True,\
        delete_pixels_polys=False, small_hole_threshold=64, solidity_filter=False, flip_outlier_edges=True, check_edges=True, plot_D_primal=True)


reading Landsat data


100%|██████████| 4/4 [00:00<00:00,  4.65it/s]


removing small holes
removing small components
running skeletonization
building graph from skeleton
finding reasonable starting and ending points on graph edges
finding nodes that are within a certain radius of the path


100%|██████████| 272/272 [00:00<00:00, 5453.53it/s]


creating linestrings for primal graph


...

100%|██████████| 274/274 [01:32<00:00,  2.96it/s]


creating directed graph


100%|██████████| 799/799 [00:06<00:00, 123.30it/s]


Found 3 source nodes: [5, 191, 38]
Found 3 sink nodes: [3, 163, 313]
After corrections, found 3 source nodes: [5, 191, 51]
After corrections, found 1 sink nodes: [3]
getting bank coordinates for main channel


100%|██████████| 799/799 [00:00<00:00, 3004.72it/s]


In [428]:
fig, ax = plt.subplots()
plot_main_banklines = True
from shapely.geometry import Polygon
plt.imshow(mndwi, extent = [left_utm_x, right_utm_x, lower_utm_y, upper_utm_y], cmap='Blues', alpha=0.5)
for i in range(2):
    if type(G_rook.nodes()[i]['bank_polygon']) == Polygon:
        x = G_rook.nodes()[i]['bank_polygon'].exterior.xy[0]
        y = G_rook.nodes()[i]['bank_polygon'].exterior.xy[1]               
    else:
        x = G_rook.nodes()[i]['bank_polygon'].xy[0]
        y = G_rook.nodes()[i]['bank_polygon'].xy[1]
    if i == 0 and plot_main_banklines:
        plt.plot(x, y, color='tab:blue', linewidth=2)
    if i == 1 and plot_main_banklines:
        plt.plot(x, y, color='tab:blue', linewidth=2)

In [425]:
fig, ax = rb.plot_im_and_lines(mndwi, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y,
    G_rook, G_primal, smoothing=False, start_x=start_x, start_y=start_y, end_x=end_x,
    end_y=end_y, plot_lines=False)

100%|██████████| 272/272 [00:00<00:00, 3243.73it/s]


In [ ]:
rb.create_channel_nw_polygon(G_rook)

### Exceedance probability plot of island areas

In [352]:
island_areas = []
channel_areas = []
total_areas = []
degrees = []
lengths = []
for node in G_rook:
    poly1 = G_rook.nodes()[node]['bank_polygon']
    poly2 = G_rook.nodes()[node]['cl_polygon']
    if poly1.area > 0 and poly2.area > poly1.area:
        island_areas.append(poly1.area)
        channel_areas.append(poly2.area - poly1.area)
        total_areas.append(poly2.area)
        degrees.append(G_rook.degree(node))
        lengths.append(poly1.length)
        
island_areas_sorted = np.sort(island_areas)
island_areas_sorted = island_areas_sorted[:-2]
exceedance = 1.-np.arange(1.,len(island_areas_sorted) + 1.)/len(island_areas_sorted)

plt.figure()
plt.loglog(island_areas_sorted, exceedance)
plt.xlabel('island area (m2)')
plt.ylabel('exceedance probability');

### Histogram of node degree distribution

In [353]:
plt.figure()
n, bins, patches = plt.hist(degrees, bins=np.arange(1.5, 25.6, 1))

plt.bar(np.arange(2,26), n)
plt.xticks(np.arange(2,26))
plt.xlabel('node degree')
plt.ylabel('count')
plt.title('number of neighbors in island neighborhood graph');

## Purus River, Brazil

The water mask was generated in Google Earth Engine.

Initial output from GEE needs to be converted to UTM coordinates before running rivabar.

Link to data files: https://utexas.box.com/s/cb6wsihdoykbti1iju57rs2cfrszqmyx

In [430]:
reload(rb)
dirname = "../data/Purus/"
fname = "L8_mndwi_Purus_2017_UTM.tif"
start_x = 363166.2
start_y = -1011937.1
end_x = 1254178.3
end_y = -528280.7

D_primal, G_rook, G_primal, mndwi, dataset, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y, xs, ys = \
        rb.extract_centerline(fname, dirname, start_x, start_y, end_x, end_y,\
        file_type='water_index', ch_belt_smooth_factor=1e8, ch_belt_half_width=2000, remove_smaller_components=True,\
        delete_pixels_polys=False, small_hole_threshold=100, solidity_filter=False, plot_D_primal=True, flip_outlier_edges=False, check_edges=False)

removing small holes
removing small components
running skeletonization
building graph from skeleton
finding reasonable starting and ending points on graph edges
finding nodes that are within a certain radius of the path


100%|██████████| 11/11 [00:00<00:00, 1096.06it/s]


creating linestrings for primal graph


13it [00:00, 14.08it/s]


start and end nodes in G_primal:
0 9
getting bank coordinates for the two main banks


...

100%|██████████| 13/13 [01:37<00:00,  7.48s/it]


creating directed graph


100%|██████████| 34/34 [00:03<00:00,  9.59it/s]


getting bank coordinates for main channel


100%|██████████| 34/34 [00:00<00:00, 50.92it/s]


### Plot width data for main channel

In [431]:
xl, yl, w1l, w2l, w, s = rb.get_channel_widths_along_path(D_primal, D_primal.graph['main_path'])
plt.figure(figsize=(12, 4))
plt.plot(s/1000, np.array(w)*30.0)
plt.xlabel('along-channel distance (km)')
plt.ylabel('channel width (m)')
plt.xlim(0, s[-1]/1000)
plt.ylim(0, 1650);

### Plot meander wavelength and channel width data

In [432]:
fig, ax = plt.subplots()
plt.imshow(mndwi, extent=[left_utm_x, right_utm_x, lower_utm_y, upper_utm_y], cmap='gray_r', alpha=0.5)
df, curv, s, loc_zero_curv, xsmooth, ysmooth = rb.analyze_width_and_wavelength(D_primal, D_primal.graph['main_path'], ax, delta_s=30.0, 
                                                                    smoothing_factor=0.25*1e8, min_sinuosity=1.1, dx=30)

### Plot a few bends in a different projection

In [433]:
reload(rb)
# need to convert the original UTM bounds of the image to the new ones:
transformer1 = pyproj.Transformer.from_crs(CRS.from_epsg(32619), CRS.from_epsg(5880), always_xy=True)
left_utm_x1, lower_utm_y1 = transformer1.transform(left_utm_x, lower_utm_y)
right_utm_x1, upper_utm_y1 = transformer1.transform(right_utm_x, upper_utm_y)

fig = plt.figure(figsize=(14,10))
ax = plt.axes(projection=ccrs.epsg(5880))
# plt.imshow(mndwi, extent=[left_utm_x1, right_utm_x1, lower_utm_y1, upper_utm_y1], cmap='gray_r', alpha=0.5)

x1 = D_primal.graph['main_channel_bank1_coords'][:,0]
y1 = D_primal.graph['main_channel_bank1_coords'][:,1]
x2 = D_primal.graph['main_channel_bank2_coords'][:,0]
y2 = D_primal.graph['main_channel_bank2_coords'][:,1]
x1s, y1s = transformer1.transform(x1, y1)
x2s, y2s = transformer1.transform(x2, y2)
plt.plot(x1s, y1s, 'k')
plt.plot(x2s, y2s, 'k')
plt.axis('equal')

# draw lat/lon gridlines:
transformer = pyproj.Transformer.from_crs(CRS.from_epsg(5880), CRS.from_epsg(4326), always_xy=True)
lonmin, latmin = transformer.transform(min(x1s), min(y1s))
lonmax, latmax = transformer.transform(max(x1s), max(y1s))
lons = rb.find_numbers_between(lonmin, lonmax, 1)
lats = rb.find_numbers_between(latmin, latmax, 1)
ax.gridlines(draw_labels=True, xlocs = mticker.FixedLocator(lons), ylocs = mticker.FixedLocator(lats), color='k', linewidth=0.5);

fig, ax2 = plt.subplots()
df, curv, s, loc_zero_curv, xsmooth, ysmooth = rb.analyze_width_and_wavelength(D_primal, D_primal.graph['main_path'], ax2, delta_s=30.0, smoothing_factor=0.25*1e8, min_sinuosity=1.1, dx=30)
xsmooth, ysmooth = transformer1.transform(xsmooth, ysmooth)
ax.plot(xsmooth, ysmooth)
ax.plot(xsmooth[loc_zero_curv], ysmooth[loc_zero_curv], 'ro', markersize=4)

ax.set_xlim(3630105.04, 3656214.98)
ax.set_ylim(9136647.83, 9154217.01)

scalebar = ScaleBar(1.0, "km", length_fraction=0.25, location='lower right', box_color="white", pad=0.5)
ax.add_artist(scalebar);

In [434]:
df.head()

,wavelengths (m),sinuosities,mean widths (m),std. dev. of widths (m),along-channel distance (km)
0,1705.567748,1.239965,124.310666,28.088273,2.801184
1,4263.379816,1.454972,134.235692,27.631182,6.206796
2,1581.067501,2.017297,131.736915,21.691657,10.055121
3,3620.877636,1.406359,147.689805,32.220466,13.784208
4,3006.874515,1.153527,151.072455,40.869887,15.964955


### Merge two files into a single TIF file (often needed when downloading files from GEE)

In [ ]:
dirname = "../data/Purus/"
fname1 = 'L8_mndwi_mosaic_Purus_2019_2020-0000000000-0000000000_UTM.tif'
fname2 = 'L8_mndwi_mosaic_Purus_2019_2020-0000000000-0000032768_UTM.tif'

from rasterio.merge import merge

# Open the first raster dataset
src1 = rasterio.open(dirname+fname1) 
meta1 = src1.meta
# Open the second raster dataset
src2 = rasterio.open(dirname+fname2)
# Merge the two arrays
merged_arr, merged_transform = merge([src1, src2])
merged_arr = merged_arr[0, :, :]
# Update the metadata with the merged shape and transform
meta1.update({
    'height': merged_arr.shape[0],
    'width': merged_arr.shape[1],
    'transform': merged_transform
})
# Write the merged raster dataset to disk
with rasterio.open(dirname + 'L8_mndwi_mosaic_Purus_2019_2020_UTM.tif', 'w', **meta1) as dst:
    dst.write(merged_arr, 1)

## Adelaide River, Australia

Water mask was generated from Planet Labs data.

Link to data file: https://utexas.box.com/s/g5mgrpw3pir8lwfnlx7nnexggg7o59an

In [ ]:
dirname = '../data/Adelaide/'
fname = 'ndwi_mosaic.tif'
start_x = 753443.4
start_y = 8598434.2
end_x = 743089.7
end_y = 8648110.2

D_primal, G_rook, G_primal, mndwi, dataset, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y, xs, ys = \
        rb.extract_centerline(fname, dirname, start_x, start_y, end_x, end_y, mndwi_threshold = 0.5,\
        file_type='water_index', ch_belt_smooth_factor=1e8, ch_belt_half_width=2000, remove_smaller_components=False,\
        delete_pixels_polys=False, small_hole_threshold=64, solidity_filter=False, plot_D_primal=True)

removing small holes
removing small components
running skeletonization
building graph from skeleton
finding reasonable starting and ending points on graph edges
finding nodes that are within a certain radius of the path


100%|██████████| 80/80 [00:00<00:00, 1974.61it/s]
0it [00:00, ?it/s]


creating linestrings for primal graph


2it [00:00,  8.40it/s]


start and end nodes in G_primal:
1 0
getting bank coordinates for the two main banks


...

100%|██████████| 2/2 [00:14<00:00,  7.15s/it]


creating directed graph


100%|██████████| 1/1 [00:00<00:00, 10.60it/s]


getting bank coordinates for main channel


100%|██████████| 28512/28512 [00:06<00:00, 4257.59it/s] 


### Plot width data for main channel

In [52]:
xl, yl, w1l, w2l, w, s = rb.get_channel_widths_along_path(D_primal, D_primal.graph['main_path'])
plt.figure(figsize=(12, 4))
plt.plot(s, np.array(w)*3.0)
plt.xlabel('along-channel distance (m)')
plt.ylabel('channel width (m)');

### Plot meander wavelength and channel width data

In [53]:
fig, ax = plt.subplots()
plt.imshow(mndwi, extent=[left_utm_x, right_utm_x, lower_utm_y, upper_utm_y], cmap='gray_r', alpha=0.5)
df, curv, s, loc_zero_curv, xsmooth, ysmooth = rb.analyze_width_and_wavelength(D_primal, D_primal.graph['main_path'], ax, 
                                                            delta_s=5, smoothing_factor=1e6, min_sinuosity=1.1, dx=30)

In [54]:
df.head()

,wavelengths (m),sinuosities,mean widths (m),std. dev. of widths (m),along-channel distance (km)
0,1702.798992,1.260533,1177.606188,53.253008,3.320376
1,1827.987198,3.536084,1103.230996,81.002739,5.472576
2,3849.473068,1.380029,1038.693355,54.884470,8.420764
3,2375.920113,1.172625,1130.309515,62.764570,11.686329
4,3174.248635,1.334377,1074.331173,65.234065,13.435510


## Lena Delta, Russia


Link to data file: https://utexas.box.com/s/4b87y3og9o40zti0cytye0ppecepncy8

In [411]:
# it takes about 45 minutes to run this
dirname = '../data/Lena/'
fname = 'lena_1_mndwi_clipped.tif'

start_x = 437535.424
start_y = 7970396.089
end_x = 522971.474
end_y = 8092767.374

D_primal, G_rook, G_primal, mndwi, dataset, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y, xs, ys = \
        rb.extract_centerline(fname, dirname, start_x, start_y, end_x, end_y,\
        file_type='water_index', flip_outlier_edges=True, mndwi_threshold=0.0, radius=1000,\
        ch_belt_smooth_factor=1e8, ch_belt_half_width=100000, remove_smaller_components=True,\
        delete_pixels_polys=False, small_hole_threshold=64, solidity_filter=False, check_edges=False)

removing small holes
removing small components
running skeletonization
building graph from skeleton
finding reasonable starting and ending points on graph edges
finding nodes that are within a certain radius of the path


100%|██████████| 1210/1210 [00:00<00:00, 3504.07it/s]


creating linestrings for primal graph


1212it [04:25,  4.56it/s]


start and end nodes in G_primal:
1 108
getting bank coordinates for the two main banks


...

100%|██████████| 1212/1212 [27:34<00:00,  1.37s/it] 


creating directed graph


100%|██████████| 3607/3607 [00:49<00:00, 73.06it/s]


getting bank coordinates for main channel


100%|██████████| 5317/5317 [00:02<00:00, 2222.87it/s]


In [ ]:
from shapely.geometry import Polygon
# these points can be obtained by using 'ginput':
points = [(494904.8165878182, 7970881.450080209),
 (502959.421306467, 7975311.482675467),
 (502355.3259525683, 7987192.024635473),
 (502153.9608346021, 7991017.961876832),
 (504368.97713223053, 7991017.961876832),
 (509000.37484545354, 7983970.182748014),
 (515444.05862037255, 7984574.278101913),
 (520880.9168054605, 7997663.010769717),
 (523297.2982210552, 8022229.555161595),
 (523095.93310308893, 8045185.178609745),
 (522693.2028671565, 8076396.771894509),
 (521686.3772773254, 8101366.04652232),
 (513229.04232274415, 8119488.90713928),
 (501549.8654807034, 8135195.386340645),
 (488461.1328128992, 8148888.2143623475),
 (471949.1931396692, 8159761.930732524),
 (453423.602286777, 8163587.867973882),
 (438925.3137932092, 8163185.137737949)]

x_utm, y_utm, ch_map = rb.get_channel_mouth_polygon(mndwi, dataset, points)
fig = rb.plot_im_and_lines(mndwi, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y, 
                G_rook, G_primal, smoothing=False, start_x=start_x, start_y=start_y, 
                end_x=end_x, end_y=end_y)
plt.plot(x_utm, y_utm, 'g', linewidth=3)
ch_poly = rb.create_channel_nw_polygon(G_rook, buffer=10, 
                ch_mouth_poly=Polygon(np.vstack((x_utm, y_utm)).T), 
                dataset=dataset, mndwi=mndwi)


In [441]:
D_primal, sources, sinks = rb.create_directed_multigraph(G_primal, \
        G_rook, xs, ys, 1, 108, flip_outlier_edges=True, check_edges=False, \
        x_utm=x_utm, y_utm=y_utm)
fig = rb.plot_im_and_lines(mndwi, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y,
        G_rook, G_primal, smoothing=False, start_x=start_x, start_y=start_y, end_x=end_x,
        end_y=end_y, plot_main_banklines=False, plot_lines=False)
ax = plt.gca()
rb.plot_graph_w_colors(D_primal, ax)

100%|██████████| 3470/3470 [00:01<00:00, 2023.25it/s]


In [417]:
# save graphs to pickle files
import pickle
with open(dirname+"lena_delta_g_rook.pkl", "wb") as f:
    pickle.dump(G_rook, f)
with open(dirname+"lena_delta_g_primal.pkl", "wb") as f:
    pickle.dump(G_primal, f)
with open(dirname+"lena_delta_d_primal.pkl", "wb") as f:
    pickle.dump(D_primal, f)

In [440]:
# load graphs from pickle files
dirname = '../data/Lena/'
with open(dirname+"lena_delta_g_rook.pkl", "rb") as f:
    G_rook = pickle.load(f)
with open(dirname+"lena_delta_g_primal.pkl", "rb") as f:
    G_primal = pickle.load(f)
with open(dirname+"lena_delta_d_primal.pkl", "rb") as f:
    D_primal = pickle.load(f)
ax = plt.gca()
rb.plot_graph_w_colors(D_primal, ax)

In [443]:
dirname = '../data/Lena/'
fname = 'lena_1_mndwi_clipped.tif'
im, dataset, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y = rb.read_and_plot_im(dirname, fname)

In [454]:
start_x = 437535.424
start_y = 7970396.089
end_x = 522971.474
end_y = 8092767.374

fig = rb.plot_im_and_lines(im, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y,
        G_rook, G_primal, smoothing=False, start_x=start_x, start_y=start_y, end_x=end_x,
        end_y=end_y, plot_main_banklines=True, plot_lines=False, plot_image=True)

100%|██████████| 1210/1210 [00:00<00:00, 2730.44it/s]


In [465]:
rb.write_shapefiles_and_graphs(G_rook, D_primal, dataset, dirname, 'Lena')